# MeanReverter Strategy Batch Optimization

这个notebook用于对多个交易对进行MeanReverter策略的批量优化。

In [28]:
# 导入必要的库...
import os
import pandas as pd
from datetime import datetime
from glob import glob
import backtrader as bt 
import optuna
import warnings
import quantstats as qs
warnings.filterwarnings('ignore')
from IPython.display import clear_output
from pathlib import Path  # 使用pathlib代替os
# 添加必要的导入
from concurrent.futures import ThreadPoolExecutor
# _data_loading_executor = ThreadPoolExecutor(max_workers=10)  # 用于数据加载的线程池

# 遍历每个交易对，但使用多进程方式
from joblib import Parallel, delayed
import numpy as np

# 在此处添加全局缓存字典，用于缓存数据加载结果和数据完整性检查结果
_data_feed_cache = {}
_data_completeness_cache = {}

In [29]:
from MeanReverter import MeanReverter

# 在 CONFIG 中添加所有需要动态配置的参数
CONFIG = {
    # 策略相关配置
    'strategy': {
        'class': MeanReverter,
        'name': MeanReverter.__name__
    },
    
    # 数据相关配置
    'selected_symbols': [
        "1000PEPEUSDT",
    ],
    'data_path': r'\\znas\Main\futures',  # 主数据路径
    'start_date': '2024-12-28',
    'end_date': '2025-03-23',
    'source_timeframe': '1m',
    'target_timeframes': ['15min'],
    
    # 文件路径配置
    'reports_path': 'reports',  # 报告输出目录
    'backup_path': 'backups',   # 常规备份目录
    'final_backup_path': 'backups_final',  # 最终备份目录
    'results_filename_template': 'optimization_results_{strategy_name}_{start_date}-{end_date}.csv',
    
    # 回测参数配置
    'commission': 0.0004,
    'initial_capital': 10000,
    # 如果需要可以添加：
    # 'trade_on_close': True,
    # 'exclusive_orders': True,
    # 'hedging': False,
    
    # 优化参数配置，根据 MeanReverter 策略的参数进行优化
    'optimization_params': {
        'frequency': range(4, 40, 2),            # 用于计算慢速 RSI 均线的周期，步长为2
        'rsiFrequency': range(4, 80, 2),           # 计算 RSI 的周期，步长为2
        'buyZoneDistance': range(1, 12, 1),        # RSI 相对于慢速 RSI 均线的折扣比例，步长为1
        'avgDownATRSum': range(1, 12, 1),          # 用于计算 ATR 累积值的周期数，步长为1
        'useAbsoluteRSIBarrier': [True, False],    # 是否使用绝对 RSI 阈值进行平仓
        'barrierLevel': range(24, 86, 2),           # RSI 阻力水平，步长为2
        'pyramiding': range(1, 10, 1)               # 最大允许加仓次数，步长为1
    },
    
    # 优化设置
    'optimization_settings': {
        'n_trials': 1000,       # 可根据需要调整试验次数
        'min_trades': 10,
        'timeout': 3600,
        'n_jobs': 30           # -1 表示使用所有 CPU 核心; 也可以设置为具体的数量
    },
}

In [30]:
def get_timeframe_params(timeframe_str):
    """
    将时间周期字符串转换为 backtrader 的 timeframe 和 compression 参数
    """
    if timeframe_str.endswith('min'):
        return (bt.TimeFrame.Minutes, int(timeframe_str.replace('min', '')))
    elif timeframe_str.endswith('H'):
        return (bt.TimeFrame.Minutes, int(timeframe_str.replace('H', '')) * 60)
    elif timeframe_str.endswith('D'):
        return (bt.TimeFrame.Days, 1)
    elif timeframe_str == '1m':
        return (bt.TimeFrame.Minutes, 1)
    else:
        raise ValueError(f"不支持的时间周期格式: {timeframe_str}")



def load_and_resample_data(symbol, start_date, end_date, source_timeframe='1m', target_timeframe='30min', data_path=r'\\znas\Main\futures'):
    """
    加载并重采样期货数据，并缓存已经重采样后的 DataFrame 以避免重复 I/O 操作
    """
    # 构造缓存键
    key = (symbol, start_date, end_date, source_timeframe, target_timeframe, data_path)
    if key in _data_feed_cache:
        # 如果缓存中有，返回新的数据馈送对象（注意拷贝，防止被修改）
        cached_df = _data_feed_cache[key]
        timeframe, compression = get_timeframe_params(target_timeframe)
        data_feed = bt.feeds.PandasData(
            dataname=cached_df.copy(),
            open='Open',
            high='High',
            low='Low',
            close='Close',
            volume='Volume',
            openinterest=-1,
            timeframe=timeframe,
            compression=compression,
            fromdate=pd.to_datetime(start_date),
            todate=pd.to_datetime(end_date)
        )
        
        # 添加clone方法，这样可以快速创建数据副本而不需要重新执行IO
        data_feed.clone = lambda: bt.feeds.PandasData(
            dataname=cached_df.copy(),
            open='Open',
            high='High',
            low='Low',
            close='Close',
            volume='Volume',
            openinterest=-1,
            timeframe=timeframe,
            compression=compression,
            fromdate=pd.to_datetime(start_date),
            todate=pd.to_datetime(end_date)
        )
        
        return data_feed
    
    # 生成日期范围
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    all_data = []
    
    # 标准化交易对名称
    formatted_symbol = symbol.replace('/', '_').replace(':', '_')
    if not formatted_symbol.endswith('USDT'):
        formatted_symbol = f"{formatted_symbol}USDT"
    
    # 顺序读取文件，不使用线程池
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        # 构建文件路径
        file_path = os.path.join(data_path, date_str, f"{date_str}_{formatted_symbol}_USDT_{source_timeframe}.csv")
        
        try:
            if os.path.exists(file_path):
                # 读取数据
                df = pd.read_csv(file_path)
                df['datetime'] = pd.to_datetime(df['datetime'])
                all_data.append(df)
            else:
                print(f"文件不存在: {file_path}")
        except Exception as e:
            print(f"读取文件出错 {file_path}: {str(e)}")
            continue
    
    if not all_data:
        raise ValueError(f"未找到 {symbol} 在指定日期范围内的数据")
    
    # 合并、排序，以及重采样数据
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values('datetime')
    combined_df.set_index('datetime', inplace=True)
    
    resampled = combined_df.resample(target_timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()  # 立即删除NaN值
    
    backtesting_df = pd.DataFrame({
        'Open': resampled['open'],
        'High': resampled['high'],
        'Low': resampled['low'],
        'Close': resampled['close'],
        'Volume': resampled['volume']
    })
    
    # 确保所有数据都是数值类型并删除任何无效值
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        backtesting_df[col] = pd.to_numeric(backtesting_df[col], errors='coerce')
    backtesting_df = backtesting_df.dropna()
    
    # 将结果缓存在全局变量中（使用拷贝，以免后续被修改）
    _data_feed_cache[key] = backtesting_df.copy()
    
    timeframe, compression = get_timeframe_params(target_timeframe)
    data_feed = bt.feeds.PandasData(
        dataname=backtesting_df,
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        openinterest=-1,
        timeframe=timeframe,
        compression=compression,
        fromdate=pd.to_datetime(start_date),
        todate=pd.to_datetime(end_date)
    )
    
    # 添加clone方法
    data_feed.clone = lambda: bt.feeds.PandasData(
        dataname=backtesting_df.copy(),
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        openinterest=-1,
        timeframe=timeframe,
        compression=compression,
        fromdate=pd.to_datetime(start_date),
        todate=pd.to_datetime(end_date)
    )
    
    return data_feed

In [31]:
def get_all_symbols(data_path, date_str):
    """获取指定日期目录下的所有交易对"""
    daily_path = os.path.join(data_path, date_str)
    if not os.path.exists(daily_path):
        return []
    
    files = glob(os.path.join(daily_path, f"{date_str}_*_USDT_1m.csv"))
    symbols = set()  # 使用 set 进行去重
    for file in files:
        filename = os.path.basename(file)
        symbol = filename.split('_')[1]
        symbols.add(symbol)
    return list(symbols)

def verify_data_completeness(symbol, start_date, end_date, data_path):
    """验证数据完整性"""
    # 构造缓存键
    key = (symbol, start_date, end_date, data_path)
    if key in _data_completeness_cache:
        return _data_completeness_cache[key]
    
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # 标准化交易对名称
    formatted_symbol = symbol.replace('/', '_').replace(':', '_')
    if not formatted_symbol.endswith('USDT'):
        formatted_symbol = f"{formatted_symbol}USDT"
    
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        file_path = os.path.join(
            data_path,
            date_str,
            f"{date_str}_{formatted_symbol}_USDT_1m.csv"  # 文件名格式保持不变
        )
        if not os.path.exists(file_path):
            print(f"文件不存在: {file_path}")
            _data_completeness_cache[key] = False
            return False
    _data_completeness_cache[key] = True
    return True

In [32]:
# 添加自定义评分函数
def custom_score(strat):
    """
    自定义评分函数 - 以最大化回报率为主要目标
    保留最低交易次数要求作为基本约束
    """
    # 获取交易次数
    trades = strat.analyzers.trades.get_analysis()
    total_trades = trades.get('total', {}).get('total', 0)
    
    # 从returns分析器获取总回报率
    returns = strat.analyzers.returns.get_analysis()
    total_return = returns.get('rtot', 0) * 100  # 转为百分比
    
    # 交易次数惩罚 - 确保策略至少有足够的交易
    min_trades = CONFIG['optimization_settings'].get('min_trades', 50)
    trade_penalty = 1.0 if total_trades >= min_trades else (total_trades / min_trades) ** 2
    
    # 最终得分简单地使用调整后的回报率
    # 交易次数不足的策略会受到严厉惩罚
    score = total_return * trade_penalty
    
    return score

In [33]:
def load_master_results(config):
    """
    加载全局优化结果文件，并提取已完成优化的组合（基于 'Symbol', 'Target Timeframe', 'Rank' 列）。
    """
    # 构造 master 文件路径
    start_clean = config['start_date'].replace("-", "")
    end_clean = config['end_date'].replace("-", "")
    master_file = os.path.join(
        config['reports_path'], 
        config['results_filename_template'].format(
            strategy_name=config['strategy']['name'],
            start_date=start_clean,
            end_date=end_clean
        )
    )
    
    if os.path.exists(master_file):
        try:
            master_df = pd.read_csv(master_file)
        except Exception as e:
            try:
                master_df = pd.read_excel(master_file)
            except Exception as e2:
                print(f"无法读取文件: {e}, {e2}")
                master_df = pd.DataFrame()
    else:
        master_df = pd.DataFrame()
    
    optimized_combinations = set()
    if not master_df.empty:
        if {'Target Timeframe', 'Symbol', 'Rank'}.issubset(master_df.columns):
            for symbol in master_df['Symbol'].unique():
                for tf in master_df['Target Timeframe'].unique():
                    rows = master_df[(master_df['Symbol'] == symbol) & (master_df['Target Timeframe'] == tf)]
                    ranks = rows['Rank'].tolist()
                    # 当存在 5 个排名且排名为 1 到 5 时认为该组合已完成优化
                    if len(ranks) == 5 and set(ranks) == set(range(1, 6)):
                        optimized_combinations.add((symbol, tf))
        else:
            print("警告: 结果文件缺少必要的列，将重新开始优化")
    else:
        print("优化结果文件为空")
        
    return master_file, master_df, optimized_combinations

def save_master_results(new_results, master_file):
    """
    将新的优化结果追加到全局结果文件中，只进行追加操作，并避免重复数据。
    """
    if not new_results:
        print("警告: 没有新的结果需要保存")
        return

    new_df = pd.DataFrame(new_results)
    
    try:
        # 如果文件不存在或为空，直接写入
        if not os.path.exists(master_file) or os.path.getsize(master_file) == 0:
            # 确保目录存在
            os.makedirs(os.path.dirname(master_file), exist_ok=True)
            new_df.to_csv(master_file, index=False)
            print(f"创建新文件并写入 {len(new_df)} 条记录")
            return
        
        # 读取现有数据
        try:
            existing_df = pd.read_csv(master_file)
            if existing_df.empty:
                # 如果文件存在但为空，直接写入新数据
                new_df.to_csv(master_file, index=False)
                print(f"文件为空，写入 {len(new_df)} 条新记录")
                return
        except pd.errors.EmptyDataError:
            # 如果文件存在但为空，直接写入新数据
            new_df.to_csv(master_file, index=False)
            print(f"文件为空，写入 {len(new_df)} 条新记录")
            return
        
        # 创建用于检查重复的键
        def create_key(row):
            return f"{row['Symbol']}_{row['Target Timeframe']}_{row['Rank']}"
        
        existing_keys = set(existing_df.apply(create_key, axis=1))
        new_df['_temp_key'] = new_df.apply(create_key, axis=1)
        
        # 过滤掉重复的记录
        truly_new_df = new_df[~new_df['_temp_key'].isin(existing_keys)]
        truly_new_df = truly_new_df.drop('_temp_key', axis=1)
        
        if len(truly_new_df) > 0:
            # 追加非重复的新数据
            truly_new_df.to_csv(master_file, mode='a', header=False, index=False)
            print(f"成功追加 {len(truly_new_df)} 条新记录（过滤掉 {len(new_df) - len(truly_new_df)} 条重复记录）")
        else:
            print("所有记录都已存在，无需追加")
            
    except Exception as e:
        print(f"处理数据时出错: {str(e)}")
        # 如果出错，尝试直接写入新文件
        try:
            backup_file = f"{master_file}.backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            if os.path.exists(master_file):
                os.rename(master_file, backup_file)
                print(f"已将问题文件备份为: {backup_file}")
            new_df.to_csv(master_file, index=False)
            print(f"已创建新文件并写入 {len(new_df)} 条记录")
        except Exception as e2:
            print(f"备份和重写文件时出错: {str(e2)}")
        return

    # 验证最终结果
    try:
        full_df = pd.read_csv(master_file)
        print(f"当前文件总共包含 {len(full_df)} 条记录")
    except Exception as e:
        print(f"验证追加结果时出错: {str(e)}")

In [34]:
def optimize_strategy(symbol, timeframe, master_file):
    """
    使用 Optuna 优化策略参数，直接保存最优的前 5 个参数组合
    """
    if not master_file:
        raise ValueError("master_file 参数不能为空")
    
    # ... 其余代码保持不变 ...
    # ... 其他代码保持不变 ...
    # 1. 预加载数据，只执行一次IO操作
    preloaded_data = load_and_resample_data(
        symbol, CONFIG['start_date'], CONFIG['end_date'],
        target_timeframe=timeframe,
        data_path=CONFIG['data_path']
    )
    
    study = optuna.create_study(
        study_name=f"{symbol}_{timeframe}",
        direction="maximize",
        storage=None  # 使用内存存储
    )
    
    print(f"开始优化 {symbol}-{timeframe}...")
    
    def objective(trial):
        try:
            params = {}
            for param_name, param_range in CONFIG['optimization_params'].items():
                if isinstance(param_range, range):
                    params[param_name] = trial.suggest_int(
                        param_name,
                        param_range.start,
                        param_range.stop - 1,
                        param_range.step
                    )
                else:
                    params[param_name] = trial.suggest_categorical(param_name, param_range)
            
            cerebro = bt.Cerebro(
                        optdatas=True,    
                        optreturn=True,   
                        runonce=True,     
                        preload=True      
            )
            data = preloaded_data.clone()
            cerebro.adddata(data)
            cerebro.addstrategy(CONFIG['strategy']['class'], **params)
            
            cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
            cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
            
            results = cerebro.run()
            strat = results[0]
            score = custom_score(strat)
            
            print(f"[{symbol}-{timeframe}] Trial {trial.number}: 参数 {params} -> 得分 {score:.2f}")
            
            return score
        except Exception as e:
            print(f"[{symbol}-{timeframe}] Trial {trial.number} 出错: {e}")
            return float('-inf')
    
    study.optimize(
        objective,
        n_trials=CONFIG['optimization_settings']['n_trials'],
        timeout=CONFIG['optimization_settings']['timeout'],
        n_jobs=CONFIG['optimization_settings'].get('n_jobs', 1),
        catch=(Exception,)
    )
    
    # 提取前 5 个最佳试验并直接保存
    completed_trials = [
        t for t in study.trials
        if t.state == optuna.trial.TrialState.COMPLETE and t.value is not None and t.value > float('-inf')
    ]
    
    if not completed_trials:
        print(f"警告: {symbol}-{timeframe} 没有有效的完成试验")
        return []
    
    top_trials = sorted(completed_trials, key=lambda t: t.value, reverse=True)[:5]
    
    # 准备保存结果
    results_to_save = []
    for rank, trial in enumerate(top_trials, 1):
        result = {
            'Symbol': symbol,
            'Target Timeframe': timeframe,
            'Rank': rank,
            'Score': trial.value,
            **trial.params  # 展开参数
        }
        results_to_save.append(result)
    
    # 直接保存结果
    save_master_results(results_to_save, master_file)
    
    print(f"[{symbol}-{timeframe}] 已保存最佳参数组合")
    return results_to_save

In [35]:


# def process_symbol_tf(symbol, tf, config):
#     """
#     针对单个交易对和指定时间周期执行策略参数优化与回测，并赋予 1~5 的排名。
#     """
#     print(f"\n开始针对 {symbol} 时间周期 {tf} 优化...")
#     # 使用已有的 optimize_strategy 函数
#     top_results = optimize_strategy(symbol, tf)
#     if not top_results:
#         print(f"警告: {symbol} 在 {tf} 时间周期下优化失败")
#         return []
    
#     processed_results = []
#     # 为每个参数组合运行回测并获取量化指标，同时赋予排名
#     for idx, res in enumerate(top_results, start=1):
#         res['Symbol'] = symbol
#         res['Target Timeframe'] = tf
#         res['Rank'] = idx
#         metrics = run_backtest_with_params(res, symbol, tf)
#         res.update(metrics)
#         processed_results.append(res)
        
#     print(f"完成 {symbol} 在 {tf} 时间周期下的优化，获得 {len(processed_results)} 个结果")
#     return processed_results



def auto_backup_results(master_file, backup_folder='backups'):
    """
    创建优化结果文件的自动备份
    """
    if not os.path.exists(master_file):
        print(f"文件不存在，无需备份: {master_file}")
        return
    
    # 创建备份目录
    os.makedirs(backup_folder, exist_ok=True)
    
    # 创建带时间戳的备份文件名
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.basename(master_file)
    backup_file = os.path.join(backup_folder, f"{filename}.{timestamp}.bak")
    
    try:
        import shutil
        shutil.copy2(master_file, backup_file)
        print(f"已自动创建备份: {backup_file}")
    except Exception as e:
        print(f"自动备份失败: {str(e)}")

def batch_optimize(config):
    """
    批量优化所有交易对和指定多个时间周期
    """

    # 使用多线程并行处理
    import threading
    from queue import Queue

    os.makedirs(config['reports_path'], exist_ok=True)
    
    master_file, master_df, optimized_combinations = load_master_results(config)
    
    # 获取交易对列表
    if config.get('selected_symbols'):
        symbols = config['selected_symbols']
    else:
        symbols = get_all_symbols(config['data_path'], config['start_date'])
    
    # 创建备份
    auto_backup_results(master_file)
    
    # 准备未完成的组合
    remaining_combinations = []
    for symbol in symbols:
        for tf in config['target_timeframes']:
            if (symbol, tf) not in optimized_combinations:
                remaining_combinations.append((symbol, tf))
    
    print(f"剩余需要优化的组合数量: {len(remaining_combinations)} 个")
    
    # 使用多线程处理
    task_queue = Queue()
    for combo in remaining_combinations:
        task_queue.put(combo)
    
    file_lock = threading.Lock()
    
    def worker():
        while not task_queue.empty():
            try:
                symbol, tf = task_queue.get(block=False)
                print(f"开始处理组合: {symbol}-{tf}")
                with file_lock:
                    optimize_strategy(symbol, tf, master_file)  # 传入 master_file
                task_queue.task_done()
            except Exception as e:
                print(f"处理任务出错: {e}")
                try:
                    task_queue.task_done()
                except:
                    pass
    
    # 创建并启动线程
    n_threads = min(30, len(remaining_combinations))
    threads = []
    for _ in range(n_threads):
        t = threading.Thread(target=worker)
        t.daemon = True
        threads.append(t)
        t.start()
    
    # 等待所有线程完成
    for t in threads:
        t.join()
    
    # 最终备份
    auto_backup_results(master_file, backup_folder='backups_final')
    print("批量优化完成")


In [36]:
if __name__ == '__main__':
    # 先清理不完整的 Optuna 数据库文件（不在最终结果 CSV 中的组合）
    # clean_incomplete_optuna_db_files(CONFIG)
    batch_optimize(CONFIG)

[I 2025-03-28 02:10:56,690] Trial 958 finished with value: 0.0 and parameters: {'frequency': 28, 'rsiFrequency': 26, 'buyZoneDistance': 1, 'avgDownATRSum': 9, 'useAbsoluteRSIBarrier': False, 'barrierLevel': 74, 'pyramiding': 1}. Best is trial 194 with value: 3.035131444116313.


[1000PEPEUSDT-15min] Trial 958: 参数 {'frequency': 28, 'rsiFrequency': 26, 'buyZoneDistance': 1, 'avgDownATRSum': 9, 'useAbsoluteRSIBarrier': False, 'barrierLevel': 74, 'pyramiding': 1} -> 得分 0.00


[I 2025-03-28 02:10:59,587] Trial 959 finished with value: 3.035131444116313 and parameters: {'frequency': 4, 'rsiFrequency': 26, 'buyZoneDistance': 1, 'avgDownATRSum': 10, 'useAbsoluteRSIBarrier': False, 'barrierLevel': 58, 'pyramiding': 1}. Best is trial 194 with value: 3.035131444116313.


[1000PEPEUSDT-15min] Trial 959: 参数 {'frequency': 4, 'rsiFrequency': 26, 'buyZoneDistance': 1, 'avgDownATRSum': 10, 'useAbsoluteRSIBarrier': False, 'barrierLevel': 58, 'pyramiding': 1} -> 得分 3.04


[I 2025-03-28 02:11:04,257] Trial 961 finished with value: 3.035131444116313 and parameters: {'frequency': 4, 'rsiFrequency': 26, 'buyZoneDistance': 1, 'avgDownATRSum': 9, 'useAbsoluteRSIBarrier': False, 'barrierLevel': 58, 'pyramiding': 1}. Best is trial 194 with value: 3.035131444116313.


...

[1000PEPEUSDT-15min] Trial 999: 参数 {'frequency': 4, 'rsiFrequency': 28, 'buyZoneDistance': 9, 'avgDownATRSum': 10, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 58, 'pyramiding': 1} -> 得分 0.00


[I 2025-03-28 02:15:04,201] Trial 995 finished with value: 0.0 and parameters: {'frequency': 4, 'rsiFrequency': 28, 'buyZoneDistance': 9, 'avgDownATRSum': 9, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 58, 'pyramiding': 1}. Best is trial 194 with value: 3.035131444116313.


[1000PEPEUSDT-15min] Trial 995: 参数 {'frequency': 4, 'rsiFrequency': 28, 'buyZoneDistance': 9, 'avgDownATRSum': 9, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 58, 'pyramiding': 1} -> 得分 0.00
[1000PEPEUSDT-15min] Trial 996: 参数 {'frequency': 4, 'rsiFrequency': 28, 'buyZoneDistance': 1, 'avgDownATRSum': 9, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 80, 'pyramiding': 1} -> 得分 0.00


[I 2025-03-28 02:15:04,434] Trial 996 finished with value: 0.0 and parameters: {'frequency': 4, 'rsiFrequency': 28, 'buyZoneDistance': 1, 'avgDownATRSum': 9, 'useAbsoluteRSIBarrier': True, 'barrierLevel': 80, 'pyramiding': 1}. Best is trial 194 with value: 3.035131444116313.


创建新文件并写入 5 条记录
[1000PEPEUSDT-15min] 已保存最佳参数组合
已自动创建备份: backups_final\optimization_results_MeanReverter_20241228-20250323.csv.20250328_021504.bak
批量优化完成
